In [1]:
%load_ext tensorboard

from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import datetime
import pathlib
import os
import gc
from core import Model
import numpy as np
import math
from itertools import permutations

os.environ['CUDA_​DEVICE_​ORDER'] = 'PCI_BUS_ID'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
#clock_start = datetime.now().strftime("%Y%m%d-%H%M%S")
clock_start = "20240804-171809"
workdir = str(pathlib.Path().resolve())


cnot_variations= np.asarray([[19,0], [23,0], [29,0] , [31,0], [37,0], [41,0], [43,0], [47,0]])
ent_variations = [(1,0), (2,0), (3,0), (5,0), (7,0), (11,0), (13,0), (17,0)]

2024-08-05 01:48:41.398203: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-05 01:48:42.047600: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:

def frontload_data():
    from tensorflow import keras
    mnist_dataset = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()
    mnist_dataset = keras.datasets.mnist
    (train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

def pool_job(input):
    this_model = Model(*input)
    this_model.set_data()
    #this_model.prep()
    this_model.prep_qrand_variant(input[-2],input[-1])
    this_model.pre_filter()
    this_model.fit()
    this_model.save_filtered()

#to prevent race condition
frontload_data()

for cnot_var in cnot_variations:
    with ProcessPoolExecutor(8) as executor:
        runner = {
                executor.submit(pool_job, (1,2,clock_start,workdir,cnot_var,m)): m for m in ent_variations
            }
        for future in as_completed(runner):
            runner.pop(future)


In [ ]:

#pool_job((1,2,clock_start,workdir,cnot_variations[0],ent_variations[0]))

In [2]:

def pool_job2(detail):
    restored_model = Model(*detail)
    restored_model.set_data()
    #restored_model.prep()
    restored_model.prep_qrand_variant(detail[-2],detail[-1])
    restored_model.load_history()
    restored_model.visuals.circuit()
    restored_model.visuals.weights()
    restored_model.visuals.relative_weights()
    restored_model.visuals.bias()
    restored_model.visuals.relative_bias()
    restored_model.visuals.data_samples()
    restored_model.visuals.relative_samples()
    restored_model.visuals.filtering()


for cnot_var in cnot_variations:
    with ProcessPoolExecutor(8) as executor:
        runner = {
                executor.submit(pool_job2, (1,2,clock_start,workdir,cnot_var,m)): m for m in ent_variations
            }
        for future in as_completed(runner):
            runner.pop(future) 

2024-08-05 01:49:10.411512: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-08-05 01:49:10.411589: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-08-05 01:49:10.413611: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-08-05 01:49:10.413757: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-08-05 01:49:10.413850: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-08-05 01:49:10.414056: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-08-05 01:49:10.414072: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-08-05 01:49:10.414212: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-08-05 01:49:10.414230: I external/local

  1/313 ━━━━━━━━━━━━━━━━━━━━ 2:00 385ms/step/step

I0000 00:00:1722822551.688864    1531 service.cc:145] XLA service 0x729048004000 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722822551.688901    1531 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-08-05 01:49:11.697772: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-05 01:49:11.748239: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1722822551.750223    1494 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1722822551.753526    1509 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1722822551.758584    1560 service.cc:145] XLA service 0x729050002c40 initi

103/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

I0000 00:00:1722822551.929461    1531 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1722822551.971728    1560 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--5-0.png


/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--2-0.png


/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--13-0.png
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--17-0.png
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--1-0.png
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--7-0.png
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--11-0.png
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/circuits/FASHIONMNISTFASHIONMNISTqcnot47-0--3-0.png
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/weights/FASHIONMNISTFASHIONMNISTqcnot47-0--5-0.gif
/workspaces/QML-QPF/multiprocessing/output/20240804-171809/visuals/weights/FASHIONMNISTFASHIONMNISTqcnot47-0--7-0.gif
/workspaces/QML-QPF/multiprocessing/output/2024

In [ ]:

def UMAP_job(detail):
    restored_model = Model(*detail)
    restored_model.set_data()
    restored_model.prep()
    restored_model.load_history()
    restored_model.visuals.manifold_umap()

for cnot_var in cnot_variations:
    {UMAP_job((1,2,clock_start,workdir,cnot_var,m)): m for m in ent_variations}